In [1]:
%run lyft_data.ipynb

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
most_popular_routes_2 = routes_by_popularity[:1000]

In [ ]:
most_popular_se = SourceDestName(most_popular_routes_2.loc[0]["start_station_name"], most_popular_routes_2.loc[0]["end_station_name"])

In [ ]:
mp_df = most_popular_se.df_filter(agg_sf)

In [ ]:
mp_fit  = mp_df.sample(frac=0.8, replace=False, random_state=5)
mp_test = mp_df[~mp_df.apply(tuple,1).isin(mp_fit.apply(tuple,1))]

In [ ]:
mp_under_1 = mp_df[mp_df["duration_sec"] < 3600]
mp_under_fit  = mp_under_1.sample(frac=0.8, replace=False, random_state=5)
mp_under_test = mp_under_1[~mp_under_1.apply(tuple,1).isin(mp_under_fit.apply(tuple,1))]

In [ ]:
mp_fit["duration_sec"].to_csv("most_popular.csv")

In [ ]:
mp_test["duration_sec"].to_csv("most_popular_test.csv")

In [ ]:
def get_duration_moments(df, n_moments = 3):
    out_list = []
    for i in range(1,n_moments+1):
        out_list.append(df["duration_sec"].apply(lambda x: x**i).mean())
    return out_list

In [ ]:
most_popular_se_50 = [ SourceDestName(most_popular_routes_2.loc[i]["start_station_name"], most_popular_routes_2.loc[i]["end_station_name"]) for i in range(50)]

In [ ]:
fits = [
    get_duration_moments(se.df_filter(agg_sf).sample(frac=0.8, replace=False, random_state=5)) for se in most_popular_se_50]

In [ ]:
sorted_duration = sorted(mp_df["duration_sec"])
p = 1/len(sorted_duration)
cdf = []
c_prob = 0
for s in sorted_duration:
    c_prob += p
    cdf.append([s, c_prob])

In [ ]:
plt.plot([x[0] for x in cdf], [x[1] for x in cdf])

In [ ]:
get_duration_moments(mp_fit, 15)

In [ ]:
fit_cdf_8 = pd.read_csv("fit_cdf_8.csv")

In [ ]:
plt.plot([x[0] for x in cdf][:12322], [x[1] for x in cdf][:12322])
plt.plot(fit_cdf_4)
plt.title("Fitted cdf for most popular route - 4 moments")

In [ ]:
def erlang_cdf(x, erlang_params):
    erlang_rate = erlang_params[0]
    erlang_k = erlang_params[1]
    acc = 0.0
    for n in range(erlang_k):
        sterm  = (erlang_rate*x)**n
        sterm *= (math.e**(-1.0*erlang_rate*x))
        sterm /= (math.factorial(n))
        acc += sterm
    return 1-acc

def exp_cdf(x, exp_params):
    exp_rate = exp_params[0]
    return 1- (math.e**(-1.0*exp_rate*x))

def get_erlang_params(duration_data):
    dur_mean = duration_data.mean()
    dur_std = duration_data.std()
    erlang_rate = dur_mean/(dur_std**2)
    erlang_k = round(dur_mean*erlang_rate)
    erlang_rate = float(erlang_k)/dur_mean
    #exp_rate = 1/dur_mean
    return [erlang_rate, erlang_k]